In [1]:
from collections import namedtuple
# neptune
import os
import numpy as np
from transformers import BertForSequenceClassification
from time import sleep
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.nn import functional as F
from models.t5 import *
from data.data import *
from data.prerpocessing import *
device="cuda"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gersa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class Args:
    def __init__(self):
        self.data_path =  "D:\Github/decepticon/Processed_New"
        self.batch_size =  16
        self.num_workers =  0
        self.pretrained_model = "t5-small"
        self.pretrained_eval_model = "iarfmoose/bert-base-cased-qa-evaluator"
        self.tokenizer_len = 32103 ## manual
        self.padding_id = 0
        self.special_tokens = ["<answer>", "<question>", "<context>"]
    def to_dict(self):
        out = dict()
        out["data_path"] = self.data_path
        out["batch_size"] = self.batch_size
        out["num_workers"] = self.num_workers
        out["pretrained_model"] = self.pretrained_model
        out["tokenizer_len"] = self.tokenizer_len
        out["padding_id"] = self.padding_id
        out["special_tokens"] = self.special_tokens
        return out

hparams=Args()

In [3]:
model = T5FinetuneForRACE.load_from_checkpoint("D:/Github/decepticon/project/checkpoints/1_01.ckpt").to(device)

In [20]:
#data_module = RaceDataModule(hparams, custom_collate_fn = RaceDataModule.t5_collate_fn)
data_module = RaceDataModule(hparams, custom_collate_fn = RaceDataModule.distractor_collate_fn)
data_module.prepare_data()
data_module.setup()

DataModule: Custom collate function is detected
SETUP: Training Dataset
SETUP: Validation Dataset
SETUP: Test Dataset


In [21]:
T5FinetuneForRACE.datamodule = data_module

In [32]:
x, y = next(iter(data_module.val_dataloader()))

In [59]:
output = model.model.generate(input_ids = x["input_ids"].to(device), 
                                     do_sample = True,
                                     top_k = 50,
                                     top_p = 0.97, ## the more - the less diverse
                                     early_stopping = True)

In [60]:
data_module.tokenizer.decode(x["input_ids"][0], skip_special_tokens= False, clean_up_tokenization_spaces = True)

'<answer> sought to eliminate monopolies in favor of competition in the market-place<question> The Sherman Antitrust Act _.<context> One thinks of princes and presidents as some of the most powerful people in the world; however, governments, elected or otherwise, sometimes have had to struggle with the financial powerhouses called tycoons. The word tycoon is relatively new to the English language. It is Chinese in origin but was given as a title to some Japanese generals. The term was brought to the United States, in the late nineteenth century, where it eventually was used to refer to magnates who acquired immense fortunes from sugar and cattle, coal and oil, rubber and steel, and railroads. Some people called these tycoons "capitals of industry" and praised them for their contributions to U.S. wealth and international reputation. Others criticized them as cruel "robber barons", who would stop at nothing in pursuit of personal wealth. The early tycoons built successful businesses, oft

In [67]:
data_module.tokenizer.decode(output[1], skip_special_tokens= True, clean_up_tokenization_spaces = True)

'need to be used on major companies to reduce monopolies'

## 